In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn

In [73]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [163]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [164]:
def TweetTemizle(Dataset):
    yedek=[]
    print ("Tweetler Temizleniyor...")

    pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
    for i in range(len(Dataset["Tweet_Değiştirilmiş"])):        
        Dataset["Tweet_Değiştirilmiş"][i]=pattern.sub('', Dataset["Tweet_Değiştirilmiş"][i])
        
    print ("Tweetler Temizlendi!")
    return Dataset

In [165]:
def Ceviri(Veri):
    metin=Veri["Tweet_Değiştirilmiş"]
    for yazi in Veri:
        Ceviri_Blob=TextBlob(yazi)
        Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')
    
    for i in range(len(Veri["Tweet_Değiştirilmiş"])):        
        Veri["Tweet_Değiştirilmiş"][i]=Eng_Blob[i]
    return Veri

In [166]:
def KelimeAyristirma(DataSet):
    data_test = DataSet.copy()
    stop = stopwords.words('turkish')
    data_test["Tweet_Değiştirilmiş"] = data_test["Tweet_Değiştirilmiş"].str.lower().str.split()
    data_test["Tweet_Değiştirilmiş"] = data_test["Tweet_Değiştirilmiş"].apply(lambda x: [item for item in x if item not in stop])

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity 

In [167]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    DataSet = pd.DataFrame()


    DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Değiştirilmiş'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [168]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr',count=100)
    
    results=[]
    for item in cursor.items(1000): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)    
    TweetTemizle(DataSet)
    DataSet.to_csv('tw.csv',index=False)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!


In [169]:
Veri=pd.read_csv('tw.csv')

Veri.head()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1598390572066258944,@Korkut1122 @stormbyy @turkdegs Abi adamlar ztn seville haritasını tanıyor yani seni yok sayıyor sen hem askari hem… https://t.co/3avYgETX4n,Abi adamlar ztn seville haritasını tanıyor yani seni yok sayıyor sen hem askari hem,2022-12-01 18:56:23+00:00,1457059581356560388,Barney Stinson;),tr
1,1598375341927501824,CEM /PERŞEMBE \nKerbela Şehidi Sultanı Sensin\nYası Matemini Tutanlar Gelsin\nGönlüm Sarayında Oturan Sensin\nKadılar K… https://t.co/f4x9vi1WXN,CEM PERŞEMBE Kerbela Şehidi Sultanı SensinYası Matemini Tutanlar GelsinGönlüm Sarayında Oturan SensinKadılar K,2022-12-01 17:55:52+00:00,1344243597944696832,dede,tr
2,1598370991297728526,@koczeynep123 @KemalSariyildi3 @sakossakk @munevveryalcin @huryalcinkaya @AdayEsat @pek20862298 @idem43370857… https://t.co/Wm4sFxBDXf,,2022-12-01 17:38:35+00:00,1344243597944696832,dede,tr
3,1598368303101116416,"#buca Babam bizi Yılmaz Oglum güldürmeyin,Cem ki dedi elaleme https://t.co/yb9LTYAfCV",Babam bizi Yılmaz Oglum güldürmeyinCem ki dedi elaleme,2022-12-01 17:27:54+00:00,878954899257610242,Abraham Jonas Eklou,tr
4,1598365830240210945,@tokcem Düşünceler duygular gibi değişkendir. Önemli olan tek unsur uzun sürsede doğruya ulaşmaktır. Cem Başkan,Düşünceler duygular gibi değişkendir Önemli olan tek unsur uzun sürsede doğruya ulaşmaktır Cem Başkan,2022-12-01 17:18:04+00:00,142575325,Bişar ÇINAR,tr


In [170]:
KelimeAyristirma(Veri)


In [171]:
savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Değiştirilmiş'].apply(getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Değiştirilmiş'].apply(getPolarity)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
0,1598390572066258944,@Korkut1122 @stormbyy @turkdegs Abi adamlar ztn seville haritasını tanıyor yani seni yok sayıyor sen hem askari hem… https://t.co/3avYgETX4n,Abi adamlar ztn seville haritasını tanıyor yani seni yok sayıyor sen hem askari hem,2022-12-01 18:56:23+00:00,1457059581356560388,Barney Stinson;),tr,0.0,0.0
1,1598375341927501824,CEM /PERŞEMBE \nKerbela Şehidi Sultanı Sensin\nYası Matemini Tutanlar Gelsin\nGönlüm Sarayında Oturan Sensin\nKadılar K… https://t.co/f4x9vi1WXN,CEM PERŞEMBE Kerbela Şehidi Sultanı SensinYası Matemini Tutanlar GelsinGönlüm Sarayında Oturan SensinKadılar K,2022-12-01 17:55:52+00:00,1344243597944696832,dede,tr,0.0,0.0
2,1598370991297728526,@koczeynep123 @KemalSariyildi3 @sakossakk @munevveryalcin @huryalcinkaya @AdayEsat @pek20862298 @idem43370857… https://t.co/Wm4sFxBDXf,,2022-12-01 17:38:35+00:00,1344243597944696832,dede,tr,0.0,0.0
3,1598368303101116416,"#buca Babam bizi Yılmaz Oglum güldürmeyin,Cem ki dedi elaleme https://t.co/yb9LTYAfCV",Babam bizi Yılmaz Oglum güldürmeyinCem ki dedi elaleme,2022-12-01 17:27:54+00:00,878954899257610242,Abraham Jonas Eklou,tr,0.0,0.0
4,1598365830240210945,@tokcem Düşünceler duygular gibi değişkendir. Önemli olan tek unsur uzun sürsede doğruya ulaşmaktır. Cem Başkan,Düşünceler duygular gibi değişkendir Önemli olan tek unsur uzun sürsede doğruya ulaşmaktır Cem Başkan,2022-12-01 17:18:04+00:00,142575325,Bişar ÇINAR,tr,0.0,0.0


In [172]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
289,1595332893177421824,@pin_trader 1000 olacak inşallah🧿,olacak inşallah🧿,2022-11-23 08:26:16+00:00,1483344989430198274,Cem-Ankara,tr,0.0,0.0
290,1595329447720357896,@DrGunerSonmez Bu uygulamayı indirin:\n\nhttps://t.co/cI5QBn28y3,Bu uygulamayı indirin,2022-11-23 08:12:34+00:00,1152886988665708544,Djem,tr,0.0,0.0
291,1595305175614427136,"@jamessonuzan Bu yorum seni linçlememe engel değil Cem, gözden düştün 🤭🥲",Bu yorum seni linçlememe engel değil Cem gözden düştün 🤭🥲,2022-11-23 06:36:07+00:00,2412709126,AKBALLARDAN BERFİN ރ,tr,0.0,0.0
292,1595294539513741313,@idyllicyoruks AYYY İMDO AĞLICAMM CEM KARACA💞,AYYY İMDO AĞLICAMM CEM KARACA💞,2022-11-23 05:53:52+00:00,1512579968743743491,nur,tr,0.0,0.0
293,1595277485117259776,@SportSantra Cem Uzan,Cem Uzan,2022-11-23 04:46:05+00:00,984492887974006784,SantaKyrox,tr,0.0,0.0
